#Vectorisation et modelisation

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from wordcloud import WordCloud

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
#import contractions


nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
DATASET_FILE = "dataset_cleaned.csv"

In [4]:
dataset_df = pd.read_csv(DATASET_FILE)
dataset_df

,text,stars,length,text_cleaned
0,I've only had food from here once and it wasn'...,1,68,food memorable panang curry balance flavor lik...
1,I will never return here again. Ever. I was ...,1,87,NOT_return ever sit booth wait dinner come scu...
2,I wish my experience was great as others. I di...,1,166,wish experience great others din wednesday nig...
3,Are the rosemary grapefruit scones supposed to...,1,81,rosemary grapefruit scone suppose taste like w...
4,Our takeout order was half wrong. Food was mis...,1,32,takeout order half wrong food miss portion siz...
...,...,...,...,...
24995,I was a loyal fan of Aroy before the ownership...,5,75,loyal fan aroy ownership change apprehensive v...
24996,Stopped here for a bite while wandering around...,5,55,stopped bite wander around faneuil hall pleasa...
24997,"A quiet place with excellent food, great music...",5,32,quiet place excellent food great music helpful...
24998,Super delicious food. Awesome vibe. I suffered...,5,41,super delicious food awesome vibe suffer disne...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,1), max_df = .75, min_df = .01) #un seuil minimale d'apparition de mots et maximale aussi
df = dataset_df[dataset_df.stars<3]


In [6]:
X = vectorizer.fit_transform(df.text_cleaned)
feature_names = vectorizer.get_feature_names()
#create a dense list
print(X.shape)
#print(df)

(10000, 868)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
X = TfidfVectorizer(ngram_range = (1,1), max_df = .7, min_df = .01)

data_doc_mx = X.fit_transform(df.text_cleaned)

data_dtm_text = pd.DataFrame(data_doc_mx.toarray(), columns=X.get_feature_names() )

data_dtm_text.index = df.index
#dataframe tf-idf
#data_dtm_text
# print(data_doc_mx)
print(X)



TfidfVectorizer(max_df=0.7, min_df=0.01)


In [8]:
### desplay les topic functions
def display_topics(model, feature_names, num_top_words,topic_names=None):
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [9]:
#run modeling on doc
nmf_model = NMF(15)
# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_text)
# Extract top words from the topic-term matrix 'H' display_topics(nmf_model, tv_noun.get_feature_names(), 5)
display_topics(nmf_model, X.get_feature_names(), 5)
print(doc_topic)
#print(nmf_model)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,



Topic  0
say, customer, tell, call, manager

Topic  1
good, taste, like, dish, sauce

Topic  2
pizza, crust, cheese, topping, slice

Topic  3
order, take, delivery, get, wrong

Topic  4
food, service, bad, slow, good

Topic  5
table, come, server, waitress, ask

Topic  6
burger, fry, bun, onion, cheese

Topic  7
wait, minute, hour, 30, 15

Topic  8
chicken, rice, fry, wing, sauce

Topic  9
bar, drink, beer, bartender, night

Topic  10
time, last, first, location, second

Topic  11
place, like, really, get, look

Topic  12
sandwich, salad, cheese, bread, lettuce

Topic  13
sushi, roll, fish, rice, salmon

Topic  14
go, back, get, win, use
[[0.         0.05879949 0.         ... 0.         0.         0.        ]
 [0.02979008 0.         0.         ... 0.00385666 0.         0.00222462]
 [0.01022991 0.03024781 0.         ... 0.         0.00535871 0.0243388 ]
 ...
 [0.03352691 0.0532386  0.         ... 0.         0.         0.00467798]
 [0.         0.03897179 0.01010715 ... 0.04128554 0.    

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
print(X.get_feature_names())

['00', '10', '100', '11', '12', '15', '18', '20', '25', '30', '40', '45', '50', 'able', 'absolutely', 'acknowledge', 'across', 'act', 'actual', 'actually', 'add', 'afternoon', 'ago', 'agree', 'ahead', 'allow', 'almost', 'alone', 'along', 'already', 'alright', 'also', 'although', 'always', 'amaze', 'amazing', 'ambiance', 'amount', 'annoy', 'another', 'answer', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'apologize', 'apparently', 'appear', 'appetizer', 'appreciate', 'area', 'around', 'arrive', 'asian', 'ask', 'assume', 'atmosphere', 'attempt', 'attention', 'attentive', 'attitude', 'austin', 'available', 'average', 'avoid', 'away', 'awesome', 'awful', 'back', 'bacon', 'bad', 'bag', 'bar', 'bartender', 'base', 'basic', 'basically', 'bathroom', 'bbq', 'bean', 'beautiful', 'become', 'beef', 'beer', 'begin', 'behind', 'believe', 'best', 'beyond', 'big', 'bill', 'birthday', 'bit', 'bite', 'black', 'bland', 'block', 'booth', 'boston', 'bother', 'bottle', 'bottom', 'bowl', 'box', 'bo

In [11]:
import pandas
liste=[]
for i in range(15):
  var='topic_'+str(i)
  liste.append(var)
liste
df_topic = pandas.DataFrame(doc_topic, columns=liste)
df_topic

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
0,0.000000,0.058799,0.000000,0.000000,0.035946,0.000000,0.001182,0.000000,0.010638,0.000000,0.000000,0.029057,0.000000,0.000000,0.000000
1,0.029790,0.000000,0.000000,0.000000,0.000000,0.025245,0.000000,0.025224,0.000475,0.004546,0.000000,0.000000,0.003857,0.000000,0.002225
2,0.010230,0.030248,0.000000,0.003348,0.016546,0.025477,0.000000,0.000000,0.057128,0.025369,0.003229,0.000000,0.000000,0.005359,0.024339
3,0.002830,0.056264,0.000236,0.000000,0.000000,0.016779,0.000676,0.000000,0.000000,0.006419,0.007872,0.021620,0.000000,0.000000,0.020332
4,0.000000,0.007107,0.000000,0.052765,0.017127,0.000000,0.000000,0.016625,0.000580,0.001213,0.000000,0.032342,0.003753,0.000472,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0.033942,0.000540,0.003527,0.000000,0.000000,0.000000,0.000000,0.002721,0.000000,0.000000,0.000000,0.003701,0.000000,0.000000
9996,0.015259,0.055137,0.064008,0.008559,0.000000,0.001732,0.001758,0.000000,0.006819,0.000000,0.014767,0.043749,0.055308,0.000000,0.011877
9997,0.033527,0.053239,0.000000,0.036482,0.021420,0.036079,0.000591,0.000000,0.000000,0.017488,0.003705,0.000000,0.000000,0.000000,0.004678
9998,0.000000,0.038972,0.010107,0.000000,0.016521,0.000000,0.000000,0.000000,0.001952,0.000000,0.000000,0.000000,0.041286,0.000000,0.024238


In [12]:
#prendre le coef max pour chaque review lié à un topic
liste=[]
for i in df_topic.columns:
  data_f=df_topic.sort_values(by=i, ascending=False)
  liste.append(data_f.index[:2].to_list())
print(liste)

[[1510, 278], [9098, 5043], [5484, 6518], [6369, 4146], [375, 6301], [9601, 6166], [6270, 7346], [5558, 4098], [9130, 1114], [5669, 9897], [9983, 7517], [920, 2490], [6384, 7258], [6286, 9041], [4079, 463]]


In [22]:
df_6=pandas.DataFrame(doc_topic)
dict={}
for i in range(15):
  top_2=df_6.nlargest(2,i,keep='all')
  top2_indexes=top_2[0].keys().to_list()
  dict[i]=df['text'][top2_indexes].to_list()
df_corr=pandas.DataFrame(dict)
df_corr
print(dict.get(5))

['Unos is actually my favorite restaurant and yet it\'s come to this--a two star review.  This particular Unos I\'ve been to more times than I can count and I\'ve had at least 3 other bad customer service experiences, but the one I had tonight takes the cake.  The guy I had twice before was so negligent that I got his name from my check and memorized it so that when I returned I\'d ask that they seat me anywhere but his area.\n\nI came in today and asked for a one person booth for one and she says they don\'t have any so they had to seat me in the back where all the tables are.  Ok, no real problem, but I was there alone, and it wasn\'t peak hours just yet; it was obvious that I just wanted to eat and enjoy a meal with minimal disturbance or distraction.  And yet, she tried to seat me DIRECTLY next to another person at a table, even though the whole space where the tables were was empty.  I said no, I\'d like to sit over there please (referring to a small table on the other side) and s

#Interpretation de Topic

In [23]:
interpretation_topi={
    'topic0' : 'TICKET PAYEMENT',
    'topic1' : 'OVERLOADED RESTAURANT',
    'topic2' : 'PIZZA/DELIVERY',
    'topic3' : 'PRICES/ONLINE-ORDERING',
    'topic4' : 'FOOD/SERVICE',
    'topic5' : 'TABLE/BOTHERING',
    'topic6' : 'SERVICE',
    'topic7' : 'TEMPS D ATTENTE',
    'topic8' : 'CHICKEN',
    'topic9' : 'BEER/BAR',
    'topic10' : 'BAD PLACE TO EAT',
    'topic11' : 'SUSHI',
    'topic12' : 'LUNCH/SANDWICH',
    'topic13' : 'AMBIANCE',
    'topic14' : 'WAITER'
}

Fichier Pickle Vectoriseur

In [24]:
import pickle
with open('vectorizer.bin', 'wb') as fichier:
  pickle.dump(X, fichier, pickle.HIGHEST_PROTOCOL)

Fichier Pickle Modèle

In [25]:
import pickle
with open('model_nmf.bin', 'wb') as fichier:
  pickle.dump(nmf_model, fichier, pickle.HIGHEST_PROTOCOL)
